   #   Building a classification model for images by using Spark

We would like to present the plan of this project.

In this notebook, we will be demonstrating how to realize this project on local. 

This approach  will be a good starting point for the next step of the project concerning deploying the project on AWS.

Once all images loaded on S3, we will be using Python scrip to interact with the data and to train + evaluate models. The training and evaluating tasks will be done using 3 services in AWS : EC2, EMR and S3.

- Step 1. Loading raw image data:
    
The aim is to extract all images form a .tar file and to put them into different folders so the task of feature extraction  can be done smotthly
    
- Step 2. Feature extraction:
 
 We will be taking avantage of using the pre-trained model VGG16 in Keras to do this job. 
 
 Notice that the VGG16 model will put an image through the trained network, so we will be ale to choose the layer from which the extraction will be started.
 
 The features will be stored in folders following the label of the sample so that classification models can interact with easily.
 
- Step3. Model training:

We will be still applying the SVM algorithm provided in MLib of SPark. We will be approaching 2 mode of clsassification : One vs One and One vs All on different classes and different size of train/test set as well.


 


 - Etape 1.  Collection des images:
    
 Le but est d'extraire toutes les images du fichier initial (sous forme de ".tar.gz") et de les ranger dans de    
 differents dossiers  de maniere a ce que l'extraction des donnees puisse se faire facilement.  
    

 - Etape 2.  Extraction des 'features' a partir des donnees image:

Nous allons tirer avantage  du modele "pre-trained" VGG16 afin d'extraire les "features"  des images.

Pour rappel, le modele VGG16 prend une image en entree  pour la passer dans son reseau comprenant plusieurs couches de 
parametres. Nous pouvons choisir la couche a partir de laquelle les parametres du modele vont s'appliquer sur l'image.  

Les "features" vont etre gardes dans les dossiers afin que les modeles Machine Learning puissent les utiliser 
corectement.

  - Etape 3. Apprentissage du modele:
        
Nous allons appliquer l'algorithme SVM de la library MLib dans le context Spark pour entrainer et evaluer des modeles. Apres avoir pousse les images sur AWS S3, nous allons interagir avec ces donnees par les scrips de Python pour entrainer les modeles. L'entrainement et l'evaluation des modeles se font en utilisant le service EMR de AWS pour beneficier Spark/ Hadoop sur AWS. 

## 1. Loading data

In [1]:
import sys
import os
import tarfile 
import re 
import numpy as np
import json
import random
from split import split_on_rawdata, split

Load all images in a folder and then extract the name of each class

In [2]:
rootPath = os.getcwd()
image_path = os.path.join(rootPath,'images')

In [3]:
tarf = tarfile.open("images.tar.gz")
filenamesIntarf = tarf.getnames()

image_list=[]
for img in filenamesIntarf[1:]:
    img = img.partition('/')[2]
    image_list.append(img)
if not os.path.exists(image_path):
    tarf.extractall()
    tarf.close()
else:
    if len(os.listdir(image_path)) == len(image_list):
        pass
tarf.close()

# get the name of each class
label=[]
for img in image_list:
    image_name=re.split('\d',img)[0].rstrip('_')
    if image_name not in label:
        label.append(image_name)
    else:
        pass  

In [4]:
print('Labels in the data set: ' + ', '.join(label))

Labels in the data set: boxer, chihuahua, pug, english_setter, english_cocker_spaniel, samoyed, Egyptian_Mau, german_shorthaired, Ragdoll, British_Shorthair, american_pit_bull_terrier, beagle, american_bulldog, shiba_inu, Abyssinian, Siamese, saint_bernard, leonberger, Maine_Coon, Birman, miniature_pinscher, Bombay, wheaten_terrier, Sphynx, scottish_terrier, newfoundland, Persian, staffordshire_bull_terrier, Bengal, keeshond, japanese_chin, great_pyrenees, Russian_Blue, havanese, basset_hound, yorkshire_terrier, pomeranian


Create 2 folders called respectivement "train" and 'test', split images into 2 parts at a thresold  move them into these folders

In [5]:
train_folder, test_folder = [image_path + str(i) for i in ['\\train','\\test']]
for f in train_folder, test_folder:
    for lb in label:
        subf=str('\\' + lb)
        if not os.path.exists(f+subf):
            os.makedirs(f+subf)
image_gr=[]
for lb in label:
    cl=[]
    image_gr.append(cl)
    for img in image_list:
        if re.split('\d',img)[0].rstrip('_')== lb:
            cl.append(img)
        else:
            pass    

Determine a thresold making the ratio between traning set and test set

For example : 0.6 will be dividing the data set with the ratio 60% train set/40% test set

In [6]:
print("Please select a tradeoff from 0 to 1 to split data into the train/test set (exp. 0.6) : ")
split_cutoff = float(input())

Please select a tradeoff from 0 to 1 to split data into the train/test set (exp. 0.6) : 
0.6


Create a function to split data

In [7]:
def split_on_rawdata(cutoff = None):
    if cutoff ==  None:
        split_cutoff = float(0.8)
    else:
        split_cutoff = cutoff    
    try:
        for gr in image_gr:
            for i in range(len(gr)):
                class_name = re.split('\d',gr[i])[0].rstrip('_')
                class_name ="\\" + class_name + "\\"
                train_images, test_images = os.listdir(train_folder + class_name), os.listdir(test_folder + class_name)
                if (len(os.listdir(image_path)) > 2):
                    if i < split_cutoff*len(gr): # for each class, take tradeoff*number of examples of this class
                        if not os.path.isfile(train_folder + class_name + gr[i]):
                            os.rename(image_path + "\\" + gr[i], train_folder + class_name + gr[i])
                    else:
                        if not os.path.isfile(test_folder + class_name + gr[i]):
                            os.rename(image_path + '\\' + gr[i], test_folder + class_name + gr[i])    
                else:
                    if len(train_images) < int(split_cutoff*len(gr)):
                        l = random.sample(test_images,int(split_cutoff*len(gr)) - len(train_images))
                        for f in l:
                            if not os.path.isfile(train_folder + class_name + f):
                                os.rename(test_folder + class_name + f, train_folder + class_name + f)
                    elif len(train_images) > int(split_cutoff*len(gr)):
                        l = random.sample(train_images, len(train_images) - int(split_cutoff*len(gr)))
                        for f in l:
                            if not os.path.isfile(test_folder + class_name + f):
                                os.rename(train_folder + class_name + f, test_folder + class_name + f)
                    else:
                        pass  
                if i == 1:
                    print(class_name, len(train_images), len(test_images))
                else:
                    pass  
    except:
        print("no image to move, please load data!")

In [8]:
split_on_rawdata(cutoff = split_cutoff )

\boxer\ 120 80
\chihuahua\ 120 80
\pug\ 120 80
\english_setter\ 120 80
\english_cocker_spaniel\ 120 80
\samoyed\ 120 80
\Egyptian_Mau\ 120 80
\german_shorthaired\ 120 80
\Ragdoll\ 120 80
\British_Shorthair\ 120 80
\american_pit_bull_terrier\ 120 80
\beagle\ 120 80
\american_bulldog\ 120 80
\shiba_inu\ 120 80
\Abyssinian\ 121 82
\Siamese\ 120 80
\saint_bernard\ 120 80
\leonberger\ 120 80
\Maine_Coon\ 120 80
\Birman\ 120 80
\miniature_pinscher\ 120 80
\Bombay\ 120 80
\wheaten_terrier\ 120 80
\Sphynx\ 120 80
\scottish_terrier\ 119 80
\newfoundland\ 120 80
\Persian\ 120 80
\staffordshire_bull_terrier\ 114 77
\Bengal\ 120 80
\keeshond\ 120 80
\japanese_chin\ 120 80
\great_pyrenees\ 120 80
\Russian_Blue\ 120 80
\havanese\ 120 80
\basset_hound\ 120 80
\yorkshire_terrier\ 120 80
\pomeranian\ 120 80


## 2. Feature extraction using a pre-trained model within Keras

- We will be using the model VGG16 to extract features from images. Let's load this model (as described in https://arxiv.org/abs/1409.1556)

In [9]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

Using TensorFlow backend.


- Initialize the model and determine the layer we will start at.

- We have decided to take the model for the layer 2.

In [10]:
base_model = VGG16(weights='imagenet')
# Model will produce the output of the 'fc2'layer which is the penultimate neural network layer
# (see the paper above for mode details)
model = Model(input=base_model.input, output=base_model.get_layer('fc2').output)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
bloc

C:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("fc...)`
  after removing the cwd from sys.path.


- Loading pyspark and diferent required modules that we will be using to build the model

In [11]:
import pyspark
from pyspark.ml.linalg import SparseVector, DenseVector
from pyspark.mllib.regression import LabeledPoint
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql import SQLContext
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.ml.classification import LogisticRegression

In [12]:
import findspark
findspark.init('C:\spark\spark-2.4.3-bin-hadoop2.7')
sc = SparkContext('local')
spark = SparkSession(sc)
os.environ.get("SPARK_HOME")

'C:\\spark\\spark-2.4.3-bin-hadoop2.7'

- Load model VGG16 as described in https://arxiv.org/abs/1409.1556

- Create a function to extract features from images

In [13]:
def extract_features(model, image_path, mode = None):
    try:
        img = image.load_img(image_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        features = model.predict(x)
        if mode == 'array':
            return features[0]
        elif mode == 'list':
            return features.tolist()[0]
        else:
            return features[0]
    except:
        pass   

- Define paths to folders and sub-folders where images have been stored.

In [14]:
base_dir = image_path
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'test')

In [15]:
label = ["".join(str(i)) for i in os.listdir(train_dir)]

In [16]:
print('Labels in the data set: ' + ', '.join(label))

Labels in the data set: Abyssinian, american_bulldog, american_pit_bull_terrier, basset_hound, beagle, Bengal, Birman, Bombay, boxer, British_Shorthair, chihuahua, Egyptian_Mau, english_cocker_spaniel, english_setter, german_shorthaired, great_pyrenees, havanese, japanese_chin, keeshond, leonberger, Maine_Coon, miniature_pinscher, newfoundland, Persian, pomeranian, pug, Ragdoll, Russian_Blue, saint_bernard, samoyed, scottish_terrier, shiba_inu, Siamese, Sphynx, staffordshire_bull_terrier, wheaten_terrier, yorkshire_terrier


- In order to store features extracted from images, we'll be creating 2 subfoldes "train" and "test" where features ,which are json format files, will be put into

In [17]:
if not os.path.exists(rootPath +'\\features'):
    for f in ['train','test']:
        os.makedirs('features\\%s' %f)

- Define paths to theses folders

In [18]:
feature_extraction_path = os.path.join(rootPath, "features")
feature_extraction_train, feature_extraction_test = [feature_extraction_path + str(i) for i in ['\\train','\\test']]

In [19]:
feature_extraction_train, feature_extraction_test = [feature_extraction_path + str(i) for i in ['\\train','\\test']]

- Create a function allowing to extract features from each image and store these feature

- Extract features of each image, save results under lists in json files

In [20]:
def extractAll():
    for folder in os.listdir(train_dir):
        if not os.path.exists(feature_extraction_train +'\\' + folder):
            os.makedirs("features\\train\\%s" %folder)
        for img in os.listdir(train_dir + "\\" + folder):
            img_path = os.path.join(train_dir + "\\" + folder,img)
            imgname = ''.join(img.split('.')[:-1]) # remove .jpeg
            features = extract_features(img_path, mode ='list')
            filepath = feature_extraction_train + "\\" + folder + '\\'+ imgname + ".json"
            if not os.path.isfile(filepath):
                with open(filepath, 'w') as out:
                    json.dump(features, out)        
    
    for folder in os.listdir(validation_dir):
        if not os.path.exists(feature_extraction_test +'\\' + folder):
            os.makedirs("features\\test\\%s" %folder)
        for img in os.listdir(validation_dir + "\\" + folder):
            img_path = os.path.join(validation_dir + "\\" + folder,img)
            imgname = ''.join(img.split('.')[:-1]) # remove .jpeg
            features = extract_features(img_path, mode = 'list')
            filepath = feature_extraction_test + "\\" + folder +'\\'+ imgname + ".json"
            if not os.path.isfile(filepath):
                with open(filepath, 'w') as out:
                    json.dump(features, out)   

- Create a function allowwing to balance between the train and the test set

In [21]:
def split(feature_extraction_train,feature_extraction_test,cutoff = None):
    if cutoff ==  None:
        split_cutoff = float(0.8)
    else:
        split_cutoff = cutoff
    
    for folder in os.listdir(feature_extraction_train):
        feature_extraction_train_folder = os.path.join(feature_extraction_train,folder)
        feature_extraction_test_folder = os.path.join(feature_extraction_test,folder)
        train_images = os.listdir(feature_extraction_train_folder)
        test_images = os.listdir(feature_extraction_test_folder)
        gr = train_images + test_images
        #D[folder] = (len(train_images), len(test_images))
        if len(train_images) < int(split_cutoff*len(gr)):
            l = random.sample(test_images,int(split_cutoff*len(gr)) - len(train_images))
            for f in l:
                if not os.path.isfile(feature_extraction_train_folder + "\\"+ f):
                    os.rename(feature_extraction_test_folder + "\\"+ f, feature_extraction_train_folder + "\\"+ f)
                    
        else :
            l = random.sample(train_images, len(train_images) - int(split_cutoff*len(gr)))
            for f in l:
                if not os.path.isfile(feature_extraction_test_folder + "\\"+ f):
                    os.rename(feature_extraction_train_folder + "\\"+ f, feature_extraction_test_folder + "\\"+ f)
    
    D = {}
    for folder in os.listdir(feature_extraction_train):
        train_images = os.listdir(feature_extraction_train_folder)
        test_images = os.listdir(feature_extraction_test_folder)
        D[folder] = (len(train_images), len(test_images)) 
        
    return D
   
         

# 3. Building model

- We can select one of two methods to train a binary classification model: One vs All or One vs One.


- As this is a simulation work, we let you the possibility to choose different parameters for building a model, such as the thresold, One vs All or One vs One based model, the classes considered as the object of a classification problem.


- Let's create 2 functions that will be enabling us to load features , transform them into a structure so the Mlib SVM can work with.

In [22]:
def loadAll(filename):
    '''
    filename = the name of an unique file , for instance'yorkshire_terrier_98.json'
    '''
    class1_train=[]
    class0_train=[]
    class1_test=[]
    class0_test=[]
    try:
        folder = re.split('\d',filename)[0].rstrip('_')
        with open(feature_extraction_train + '\\' + folder + '\\' + filename) as json_data:
            ctr = json.load(json_data) 
            if filename.startswith(class1):
                class1_train.append(ctr)
                
            else:
                class0_train.append(ctr)
    except:
        folder = re.split('\d',filename)[0].rstrip('_')
        with open(feature_extraction_test + '\\' + folder + '\\' + filename) as json_data:
            ctr = json.load(json_data) 
            if filename.startswith(class1):
                class1_test.append(ctr)
                
            else:
                class0_test.append(ctr)
                
    return filename,class1_train, class0_train, class1_test, class0_test

In [23]:
def convertAll():
    json_trainfilenames = []
    json_testfilenames = []
    for folder in os.listdir(feature_extraction_train):
        feature_extraction_train_folder = os.path.join(feature_extraction_train,folder)
        feature_extraction_test_folder = os.path.join(feature_extraction_test,folder)
        trainfiles, testfiles = os.listdir(feature_extraction_train_folder), os.listdir(feature_extraction_test_folder)
        if trainmode.lower() == 'yes':
            json_trainfilenames.append(trainfiles[::2])
            json_testfilenames.append(testfiles[::2])
            
        else:
            if (folder == class1) or (folder == class0) :
                json_trainfilenames.append(trainfiles[::2])
                json_testfilenames.append(testfiles[::2])
            else:
                pass 
    json_trainfilenames = [''.join(n) for i in range(len(json_trainfilenames)) for n in json_trainfilenames[i]]
    json_testfilenames = [''.join(n) for i in range(len(json_testfilenames)) for n in json_testfilenames[i]]
    
    # we are going to use RDD oject to handle features
    rdd_trainfilenames = sc.parallelize(json_trainfilenames)
    rdd_testfilenames = sc.parallelize(json_testfilenames)    
    trainRDD = rdd_trainfilenames.map(loadAll)
    testRDD = rdd_testfilenames.map(loadAll)
    train_features = trainRDD.collect()
    test_features = testRDD.collect()
    
    class1_train = [train_features[i][1][0] for i in range(len(train_features))
                if ((len(train_features[i][1]) !=0) and (train_features[i][1][0] is not None))]# remove Null and NoneType elements
    class0_train = [train_features[i][2][0] for i in range(len(train_features))
                    if ((len(train_features[i][2]) !=0) and (train_features[i][2][0] is not None))]
    class1_test = [test_features[i][3][0] for i in range(len(test_features))
                   if ((len(test_features[i][3]) !=0) and (test_features[i][3][0] is not None))]
    class0_test = [test_features[i][4][0] for i in range(len(test_features))
                   if ((len(test_features[i][4]) !=0) and (test_features[i][4][0] is not None))]
    return class1_train, class0_train, class1_test, class0_test

- Define model parameters

In [24]:
print("Are you looking to train a model based on method 'One vs All'? ")
print("\nIf so,  please enter 'Yes', otherwise enter 'No' ")
trainmode = str(input())

Are you looking to train a model based on method 'One vs All'? 

If so,  please enter 'Yes', otherwise enter 'No' 
no


In [25]:
print("Please choose one of the following labels as the target class : \n" +'\n' + ','.join(label))
print('')
class1 = str(input())

Please choose one of the following labels as the target class : 

Abyssinian,american_bulldog,american_pit_bull_terrier,basset_hound,beagle,Bengal,Birman,Bombay,boxer,British_Shorthair,chihuahua,Egyptian_Mau,english_cocker_spaniel,english_setter,german_shorthaired,great_pyrenees,havanese,japanese_chin,keeshond,leonberger,Maine_Coon,miniature_pinscher,newfoundland,Persian,pomeranian,pug,Ragdoll,Russian_Blue,saint_bernard,samoyed,scottish_terrier,shiba_inu,Siamese,Sphynx,staffordshire_bull_terrier,wheaten_terrier,yorkshire_terrier

Siamese


In [26]:
if trainmode.lower() == 'no':
    print('')
    class0 = str(input())
else:
    pass


Sphynx


In [27]:
print("Please select a tradeoff from 0 to 1 to split data into the train/test set (exp. 0.6) : ")
split_cutoff = float(input())

Please select a tradeoff from 0 to 1 to split data into the train/test set (exp. 0.6) : 
0.6


- Split data into train set and test set

In [28]:
spl = split(feature_extraction_train,feature_extraction_test,cutoff = split_cutoff)

In [29]:
print(class1,spl[class1])
print(class0, spl[class0])

Siamese (120, 80)


- Transform and put features to the related class before feeding to a model

In [30]:
inputs = convertAll()

In [31]:
class1_train, class0_train, class1_test, class0_test = inputs[0], inputs[1], inputs[2], inputs[3]

In [32]:
len(class0_test)

40

## 3.1 SVMWithSGD ( mlib.Classification)

- In order to train a SVM model, we need first to handle original inputs , there by these can be explored by the model.

- We will be using the function Labeledpoint to do that.

In [33]:
lbtrainvec1 = [LabeledPoint(1,class1_train[i]) for i in range(len(class1_train))]
lbtrainvec0 = [LabeledPoint(0,class0_train[i]) for i in range(len(class0_train))]

lbtestvec1 = [LabeledPoint(1,class1_test[i]) for i in range(len(class1_test))]
lbtestvec0 = [LabeledPoint(0,class0_test[i]) for i in range(len(class0_test))]
    
lbtrainvec = lbtrainvec1 + lbtrainvec0 
lbtestvec = lbtestvec1 + lbtestvec0

In [34]:
model_svm = SVMWithSGD.train(sc.parallelize(lbtrainvec), iterations=100, regParam = 0.1, regType = 'l2')

In [35]:
prediction_svm = model_svm.predict(sc.parallelize(class1_test + class0_test)).collect()

In [36]:
prediction_svm=np.array(prediction_svm)

In [37]:
ytest = np.append(np.ones(len(class1_test)),np.zeros(len(class0_test)))

In [38]:
if trainmode.lower() == 'yes':
    print("\nTrained a SVM model based on method 'OnevsAll'")
    print("\nLabel 1 %s ; Lablel 0 : all other classses" % class1)
    print('\nTrain/Test: {}/{}'.format(split_cutoff, round((1-split_cutoff),2)))
else:
    print("\nTrained a SVM model based on method 'OnevsOne'")
    print("\nLabel 1: {} ; Lablel 0: {}".format(class1,class0))
    print('\nTrain/Test: {}/{}'.format(split_cutoff, round((1-split_cutoff),2)))
    
print ('\nAccuracy : %.2f' % float((np.dot(ytest,prediction_svm.T) + np.dot(1-ytest,1-prediction_svm.T))/float(ytest.size)*100) + '%')


Trained a SVM model based on method 'OnevsOne'

Label 1: Siamese ; Lablel 0: Sphynx

Train/Test: 0.6/0.4

Accuracy : 98.75%


Execute the below cde if need to save the model

The below code can be used to make predictions 

# 3.3 Put all together

We are going to simulate the training task with different parameters such as names of classes and levels of cutoff.

In [39]:
random.seed(1)
cl = ['Ragdoll', 'Siamese','Persian', 'Egyptian_Mau' ]
c1 =[]
c0 = []
train_test = []
accuracy = []
for split_cutoff in [0.3, 0.6, 0.9]:
    spl = split(feature_extraction_train,feature_extraction_test, split_cutoff)
    for j in range(len(cl)):
        class1 = cl[j]
        i = 1
        for class0 in cl[j+1:]:
            print("\niteration %d" %i)
            print(class1,spl[class1])
            print(class0, spl[class0])
            train_mode = 'no'
            inputs = convertAll()
            class1_train, class0_train, class1_test, class0_test = inputs[0], inputs[1], inputs[2], inputs[3]
            lbtrainvec1 = [LabeledPoint(1,class1_train[i]) for i in range(len(class1_train))]
            lbtrainvec0 = [LabeledPoint(0,class0_train[i]) for i in range(len(class0_train))]
            lbtestvec1 = [LabeledPoint(1,class1_test[i]) for i in range(len(class1_test))]
            lbtestvec0 = [LabeledPoint(0,class0_test[i]) for i in range(len(class0_test))]
            lbtrainvec = lbtrainvec1 + lbtrainvec0 
            lbtestvec = lbtestvec1 + lbtestvec0
            model_svm = SVMWithSGD.train(sc.parallelize(lbtrainvec), iterations=100, regParam = 0.1, regType = 'l2')
            prediction_svm = model_svm.predict(sc.parallelize(class1_test + class0_test)).collect()
            prediction_svm=np.array(prediction_svm)
            ytest = np.append(np.ones(len(class1_test)),np.zeros(len(class0_test)))
            if trainmode.lower() == 'yes':
                print("Trained a SVM model based on method 'OnevsAll'")
                print("Label 1 %s ; Lablel 0 : all other classses" % class1)
                print('Train/Test: {}/{}'.format(split_cutoff, round((1-split_cutoff),2)))
            else:
                print("Trained a SVM model based on method 'OnevsOne'")
                print("Label 1: {} ; Lablel 0: {}".format(class1,class0))
                print('Train/Test: {}/{}'.format(split_cutoff, round((1-split_cutoff),2)))
            acc = '%.2f' %float((np.dot(ytest,prediction_svm.T) + np.dot(1-ytest,1-prediction_svm.T))/float(ytest.size)*100) 
            c1.append(class1)
            c0.append(class0)
            train_test.append('{}/{}'.format(split_cutoff, round((1-split_cutoff),2)))
            accuracy.append(acc)

            print ('Accuracy : %s' % acc + '%')
        i+=1



iteration 1
Ragdoll (60, 140)
Siamese (60, 140)
Trained a SVM model based on method 'OnevsOne'
Label 1: Ragdoll ; Lablel 0: Siamese
Train/Test: 0.3/0.7
Accuracy : 97.14%

iteration 1
Ragdoll (60, 140)
Persian (60, 140)
Trained a SVM model based on method 'OnevsOne'
Label 1: Ragdoll ; Lablel 0: Persian
Train/Test: 0.3/0.7
Accuracy : 91.43%

iteration 1
Ragdoll (60, 140)
Egyptian_Mau (60, 140)
Trained a SVM model based on method 'OnevsOne'
Label 1: Ragdoll ; Lablel 0: Egyptian_Mau
Train/Test: 0.3/0.7
Accuracy : 100.00%

iteration 1
Siamese (60, 140)
Persian (60, 140)
Trained a SVM model based on method 'OnevsOne'
Label 1: Siamese ; Lablel 0: Persian
Train/Test: 0.3/0.7
Accuracy : 99.29%

iteration 1
Siamese (60, 140)
Egyptian_Mau (60, 140)
Trained a SVM model based on method 'OnevsOne'
Label 1: Siamese ; Lablel 0: Egyptian_Mau
Train/Test: 0.3/0.7
Accuracy : 98.57%

iteration 1
Persian (60, 140)
Egyptian_Mau (60, 140)
Trained a SVM model based on method 'OnevsOne'
Label 1: Persian ; Labl

In [40]:
import pandas as pd

table = pd.DataFrame(list(zip(c1,c0,train_test,accuracy)), columns = ['class1','class0','train/test','acc(%)'])
table = table.pivot_table(index = ['class1','class0'], columns = 'train/test', values ='acc(%)', aggfunc='first').reset_index()
table.to_csv('result.csv', index = False)
pd.read_csv('result.csv')

,class1,class0,0.3/0.7,0.6/0.4,0.9/0.1
0,Persian,Egyptian_Mau,100.00,98.75,100.0
1,Ragdoll,Egyptian_Mau,100.00,100.00,100.0
2,Ragdoll,Persian,91.43,91.25,90.0
3,Ragdoll,Siamese,97.14,100.00,95.0
4,Siamese,Egyptian_Mau,98.57,100.00,100.0
5,Siamese,Persian,99.29,100.00,100.0


In [42]:
sc.stop()

## 3.2 Logistic Regression ( ml.Classification)

We would like to try out to implement a Logistic Regression taking inputs that need to be stored by data frame in spark context ( when using ml.classification and not mlib.classification).

We will be using the function DenseVector  to turn data lists into vectors in a spark context, then the function  createDataFrame to make dataframes.

In [43]:
sc = SparkContext('local')
spark = SparkSession(sc)

In [44]:
trainvec1 = [(1,DenseVector(class1_train[i])) for i in range(len(class1_train))]
trainvec0 = [(0,DenseVector(class0_train[i])) for i in range(len(class0_train))]
trainvec = trainvec1 + trainvec0
testvec1 = [(1,DenseVector(class1_test[i])) for i in range(len(class1_test))]
testvec0 = [(0,DenseVector(class0_test[i])) for i in range(len(class0_test))]
testvec= testvec1 + testvec0

In [45]:
traindf = spark.createDataFrame(trainvec, ["label", "features"])

testdf = spark.createDataFrame(testvec, ["label", "features"])

lr = LogisticRegression(maxIter=100, regParam=0.01)

Learn a LogisticRegression model. This uses the parameters stored in lr.

In [46]:
model = lr.fit(traindf)

prediction = model.transform(testdf)

out_lr = prediction.select("label","probability", "prediction").collect()

In [47]:
for row in out_lr[::20]:
    print("label=%s -> prob=%s, prediction=%s"
          % (row.label, row.probability, row.prediction))

Y=[]
pred = []
for row in out_lr:
    Y.append(row[0])
    pred.append(row[2])
Y = np.array(Y)
pred = np.array(pred)

label=1 -> prob=[2.2305797509017952e-06,0.999997769420249], prediction=1.0


In [48]:
if trainmode.lower() == 'yes':
    print("trained a Logistic regression model based on method 'OnevsAll'")
    print("label 1 %s  lablel 2 : all other classses" % class1)
else:
    print("trained a Logistic regression model based on method 'OnevsOne'")
    print("label 1: {}  lablel 2: {}".format(class1,class0))
print('Accuracy: %.2f' % float((np.dot(Y,pred.T) + np.dot(1-Y,1-pred.T))/float(Y.size)*100) + '%')

trained a Logistic regression model based on method 'OnevsOne'
label 1: Egyptian_Mau  lablel 2: Egyptian_Mau
Accuracy: 100.00%


In [49]:
sc.stop()